In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Temporal foundation

This section establishes the temporal foundation of our model. It creates a 96-week period, starting on April 1, 2024, with every data point anchored to a Friday. This ensures all subsequent cash flows align with standard weekly business cycles.

In [2]:
weeks = 96
dates = pd.date_range(start='2024-04-01', periods=weeks, freq='W-FRI')
df = pd.DataFrame(index=range(weeks))
df['Week'] = dates
df.head()

,Week
0,2024-04-05
1,2024-04-12
2,2024-04-19
3,2024-04-26
4,2024-05-03


## Revenue

This section builds the inflow mechanics of our model by layering three distinct revenue profiles. It simulates a diversified income stream by combining 1. highly predictable weekly receipts, 2. stable semi-monthly receipts, and 3. highly volatile weekly receipts. This variety allows us to test how different cash cadences, both steady and erratic, interact with our fixed and variable expenses.

### 1. Revenue A - highly predictable weekly receipts

In [3]:
df['Revenue A'] = np.round(np.random.normal(200000, 20000, size=weeks).clip(min=150000), 2)
df.head()

,Week,Revenue A
0,2024-04-05,191574.28
1,2024-04-12,195821.67
2,2024-04-19,195801.01
3,2024-04-26,174244.88
4,2024-05-03,205092.08


### 2. Revenue B - stable semi-monthly receipts

In [4]:
df['Revenue B'] = np.round(0.0, 2)
semi_monthly = False

for i in range(1, len(df)):

    if i == len(df) - 1:
        semi_monthly = True
    elif df.loc[i-1, 'Week'].day < 15 and df.loc[i, 'Week'].day >= 15:
        semi_monthly = True
    elif df.loc[i-1, 'Week'].month != df.loc[i, 'Week'].month:
        semi_monthly = True

    if semi_monthly == True:
        df.at[i, 'Revenue B'] = np.round(np.random.normal(200000, 20000, size=1).clip(min=150000), 2)
        semi_monthly = False

df.head()

,Week,Revenue A,Revenue B
0,2024-04-05,191574.28,0.00
1,2024-04-12,195821.67,0.00
2,2024-04-19,195801.01,200352.59
3,2024-04-26,174244.88,0.00
4,2024-05-03,205092.08,198369.37


### 3. Revenue C - highly volatile weekly receipts

In [5]:
df['Revenue C'] = np.round(np.random.lognormal(mean=11, sigma=0.8, size=weeks), 2)
df.head()

,Week,Revenue A,Revenue B,Revenue C
0,2024-04-05,191574.28,0.00,271379.02
1,2024-04-12,195821.67,0.00,52240.56
2,2024-04-19,195801.01,200352.59,86280.26
3,2024-04-26,174244.88,0.00,33080.78
4,2024-05-03,205092.08,198369.37,86939.90


## Expense

This section defines the outflow structure of our model. It separates expenses into two main groups: compensation and non-compensation. 

### 1. Compensation

By including different payment schedules, like biweekly payroll, payroll taxes and pension, monthly benefits and other government payments such as employer health tax (EHT), we can accurately see how these various costs hit the cash balance at different times.

#### Biweekly payroll, payroll taxes and pension

Biweekly Payroll = (Base + OT) x (1 - Tax Rate)

In [6]:
PayrollBase = 200000
PayrollOT = np.random.gamma(shape=2, scale=10000, size=weeks)
PayrollTaxBaseRate = 0.3
PensionBaseRate = 0.1
variability = np.random.uniform(0.95, 1.05, size=weeks) * (df.index.to_numpy() % 2)
df['Payroll'] = -np.round((1 - PayrollTaxBaseRate) * (PayrollBase + PayrollOT) * variability, 2)
variability = np.random.uniform(0.95, 1.05, size=weeks) * (df.index.to_numpy() % 2)
df['Payroll Taxes'] = -np.round(PayrollTaxBaseRate * (PayrollBase + PayrollOT) * variability, 2)
variability = np.random.uniform(0.95, 1.05, size=weeks) * (df.index.to_numpy() % 2)
df['Pension Contribution'] = -np.round(PensionBaseRate * (PayrollBase + PayrollOT) * variability, 2)
df.head()

,Week,Revenue A,Revenue B,Revenue C,Payroll,Payroll Taxes,Pension Contribution
0,2024-04-05,191574.28,0.00,271379.02,-0.00,-0.00,-0.00
1,2024-04-12,195821.67,0.00,52240.56,-167490.77,-72578.89,-24116.45
2,2024-04-19,195801.01,200352.59,86280.26,-0.00,-0.00,-0.00
3,2024-04-26,174244.88,0.00,33080.78,-145776.70,-58654.46,-19782.04
4,2024-05-03,205092.08,198369.37,86939.90,-0.00,-0.00,-0.00


#### Monthly benefits and other government payments such as employer health tax (EHT)

In [7]:
BenefitsBaseRate = 0.08
EHTBaseRate = 0.02
running_payroll_sum = 0
df['Benefits Contribution'] = np.round(-0.0, 2)
df['EHT'] = np.round(-0.0, 2)
running_payroll_sum = df.loc[0, 'Payroll']
monthly = False

for i in range(1, len(df)):
    current_payroll = df.loc[i, 'Payroll']
    running_payroll_sum += current_payroll

    if i == len(df) - 1:
        monthly = True
    elif df.loc[i-1, 'Week'].month != df.loc[i, 'Week'].month:
        monthly = True

    if monthly == True:
        variability = np.random.uniform(0.95, 1.05)
        df.at[i, 'Benefits Contribution'] = np.round(BenefitsBaseRate * variability * running_payroll_sum, 2)
        variability = np.random.uniform(0.95, 1.05)
        df.at[i, 'EHT'] = np.round(EHTBaseRate * variability * running_payroll_sum, 2)
        running_payroll_sum = 0
        monthly = False

df.head()

,Week,Revenue A,Revenue B,Revenue C,Payroll,Payroll Taxes,Pension Contribution,Benefits Contribution,EHT
0,2024-04-05,191574.28,0.00,271379.02,-0.00,-0.00,-0.00,-0.00,-0.00
1,2024-04-12,195821.67,0.00,52240.56,-167490.77,-72578.89,-24116.45,-0.00,-0.00
2,2024-04-19,195801.01,200352.59,86280.26,-0.00,-0.00,-0.00,-0.00,-0.00
3,2024-04-26,174244.88,0.00,33080.78,-145776.70,-58654.46,-19782.04,-0.00,-0.00
4,2024-05-03,205092.08,198369.37,86939.90,-0.00,-0.00,-0.00,-24158.56,-6457.99


### 2. Non-Compensation

For the non-compensation portion of our model, we can focus on categories that represent the "keeping the lights on" costs of a business. These typically fall into Operating Expenses (OpEx) and Capital Expenditures (CapEx).

#### Operating Expenses (OpEx)

There are predictable fixed operating expenses and variable operating expenses that fluctuate based on business activity or random needs.

In [8]:
df['Fixed OpEx'] = -np.round(np.random.normal(150000, 10000, weeks).clip(min=100000), 2)
df['Variable OpEx'] = -np.round(np.random.lognormal(mean=11, sigma=0.8, size=weeks), 2)
df.head()

,Week,Revenue A,Revenue B,Revenue C,Payroll,Payroll Taxes,Pension Contribution,Benefits Contribution,EHT,Fixed OpEx,Variable OpEx
0,2024-04-05,191574.28,0.00,271379.02,-0.00,-0.00,-0.00,-0.00,-0.00,-155113.83,-48680.96
1,2024-04-12,195821.67,0.00,52240.56,-167490.77,-72578.89,-24116.45,-0.00,-0.00,-170996.20,-58481.19
2,2024-04-19,195801.01,200352.59,86280.26,-0.00,-0.00,-0.00,-0.00,-0.00,-158036.76,-92200.20
3,2024-04-26,174244.88,0.00,33080.78,-145776.70,-58654.46,-19782.04,-0.00,-0.00,-154775.76,-60645.79
4,2024-05-03,205092.08,198369.37,86939.90,-0.00,-0.00,-0.00,-24158.56,-6457.99,-131696.06,-147922.55


#### Capital Expenditures (CapEx)

These are random shock events.

In [9]:
shock = 1 if np.random.random() < 0.05 else 0
df['CapEx'] = -np.round(shock * np.random.uniform(200000, 800000, size=weeks).clip(min=100000), 2)
df.head()

,Week,Revenue A,Revenue B,Revenue C,Payroll,Payroll Taxes,Pension Contribution,Benefits Contribution,EHT,Fixed OpEx,Variable OpEx,CapEx
0,2024-04-05,191574.28,0.00,271379.02,-0.00,-0.00,-0.00,-0.00,-0.00,-155113.83,-48680.96,-0.0
1,2024-04-12,195821.67,0.00,52240.56,-167490.77,-72578.89,-24116.45,-0.00,-0.00,-170996.20,-58481.19,-0.0
2,2024-04-19,195801.01,200352.59,86280.26,-0.00,-0.00,-0.00,-0.00,-0.00,-158036.76,-92200.20,-0.0
3,2024-04-26,174244.88,0.00,33080.78,-145776.70,-58654.46,-19782.04,-0.00,-0.00,-154775.76,-60645.79,-0.0
4,2024-05-03,205092.08,198369.37,86939.90,-0.00,-0.00,-0.00,-24158.56,-6457.99,-131696.06,-147922.55,-0.0


## Weekly Ending Balances

Let's assume the beginning balance is 1 million.

In [10]:
beginning_balance = 1000000
df['Ending Balance'] = np.round(beginning_balance + df.iloc[:, 1:].cumsum().sum(axis=1), 2)
df.head()

,Week,Revenue A,Revenue B,Revenue C,Payroll,Payroll Taxes,Pension Contribution,Benefits Contribution,EHT,Fixed OpEx,Variable OpEx,CapEx,Ending Balance
0,2024-04-05,191574.28,0.00,271379.02,-0.00,-0.00,-0.00,-0.00,-0.00,-155113.83,-48680.96,-0.0,1259158.51
1,2024-04-12,195821.67,0.00,52240.56,-167490.77,-72578.89,-24116.45,-0.00,-0.00,-170996.20,-58481.19,-0.0,1013557.24
2,2024-04-19,195801.01,200352.59,86280.26,-0.00,-0.00,-0.00,-0.00,-0.00,-158036.76,-92200.20,-0.0,1245754.14
3,2024-04-26,174244.88,0.00,33080.78,-145776.70,-58654.46,-19782.04,-0.00,-0.00,-154775.76,-60645.79,-0.0,1013445.05
4,2024-05-03,205092.08,198369.37,86939.90,-0.00,-0.00,-0.00,-24158.56,-6457.99,-131696.06,-147922.55,-0.0,1193611.24


In [11]:
print("The ending balance of the 96-week period is " + str(np.round(df.loc[weeks-1:, 'Ending Balance'].values[0], 2)) + ".")

The ending balance of the 96-week period is 2425296.68.


## Export

In [12]:
df.to_csv('data v2.csv')